In [ ]:
%env OPENAI_API_KEY="keyhere"

In [7]:
import logging

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

from kagent.tools.argo import (
    # Installation & Verification
    VerifyArgoRolloutsControllerInstall,
    VerifyKubectlPluginInstall,
    CheckPluginLogsTool,
    VerifyGatewayPluginTool,
    # Resource Generation
    GenerateResource,
)

from kagent.tools.k8s import (
    ApplyManifest,
    GetResources,
    GetResourceYAML,
    CreateResource,
    PatchResource,
    DeleteResource,
    GetAvailableAPIResources,
    GetPodLogs,
    DescribeResource,
)

from kagent.tools.docs import Config, QueryTool

# Model configuration
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
)

# Documentation agent configuration
docs_config = Config(
    docs_path="",  # empty string means the database will be downloaded
)

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks that can be executed by the team members. DO NOT MAKE UP ADDITIONAL AND UNNECESSARY SUBTASKS.
    Your team members are:
        argo_agent: Performs Argo Rollouts tasks. This is only responsible for validating the installation and checking the configuration of the Argo Rollouts controller. It does not apply the config.
        k8s_agent: Performs Kubernetes tasks, including checking resources in the cluster. It should not apply routing resources, only focus on the installation and configuration of the Argo Rollouts controller.

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

# Define tool groups for better organization
k8s_tools = [
    CreateResource(),
    GetResources(),
    ApplyManifest(),
    PatchResource(),
    DeleteResource(),
    GetResourceYAML(),
    GetAvailableAPIResources(),
    DescribeResource(),
    GetPodLogs(),
]

argo_setup_tools = [
    VerifyKubectlPluginInstall(),
    VerifyArgoRolloutsControllerInstall(),
    CheckPluginLogsTool(),
    VerifyGatewayPluginTool(),
    GenerateResource(),
]

K8S_SYSTEM_MESSAGE = """You are a Kubernetes specialist agent responsible for cluster operations and resource management.

Key Responsibilities:
1. Resource Management
   - Monitor and manage Kubernetes resources (pods, services, deployments, configmaps, custom defined resources)
   - Validate Kubernetes manifests
   - Ensure proper resource state and health

2. Cluster Operations
   - Verify cluster connectivity and health
   - Monitor resource utilization
   - Report on cluster state

Guidelines:
- Always verify resource existence before operations
- Use proper error handling for kubectl operations
- Provide clear feedback on operation status
- If no explicit tasks remain, return TERMINATE

Example interactions:
- "Get pods in namespace": List all pods with their status
- "Check service": Verify service existence and endpoints
"""

ARGO_SYSTEM_MESSAGE = f"""
You are an Argo Rollouts installation verification specialist. 
Your primary focus is checking the proper installation and setup of Argo Rollouts 
components and dependencies. You do not create any resources.

Key Responsibilities:
1. Verification Checklist:
   - Base Components:
     - Check kubectl argo rollouts plugin presence
     - Verify controller installation status. Use the argo tool to check this.
     - Validate Argo Rollouts CRDs (argoproj.io) are installed. Use the k8s_agent to check this.
   - Traffic Management:
     - Confirm presence of supported traffic provider in the cluster that supports Argo Rollouts (Istio or Kubernetes Gateway API). If one is not present, suggest instructions for installing Kubernetes Gateway APIs and a supported implementation such as Istio.
     - Verify Gateway API installation (if using KubernetesGateway API) and CRDs (gateway.networking.k8s.io). Use the k8s_agent to check this. If not present, suggest instructions for setup. 
     - Validate K8s Gateway API configuration (if using Gateway API):
       - Check Gateway API configuration in the configmap in the argo-rollouts namespace. If not present, the tool will create it. If the configmap was created, rollout the argo-rollouts controller.
       - Validate Gateway class existence. Use the k8s_agent to check this.
   - Security & Permissions:
     - Verify RBAC permissions based on the traffic provider being used. Use the k8s_agent to check this.   

2. Status Validation:
   - Check if the Argo Rollouts controller is running. Use the k8s_agent to check this.
   - If using K8s Gateway API and the configmap is present, check the controller logs for "Downloading plugin argoproj-labs/gatewayAPI"
   
If everything is checked, inform the user they are ready to create a Rollout resources.

Best Practices:
- Always verify prerequisites before checking specific resources. 
- Provide step-by-step guidance for complex tasks
- Ensure proper validation at each stage
- Document any configuration changes
"""

DOCS_SYSTEM_MESSAGE = """You are a CNCF documentation specialist providing accurate information from official sources.

Key Responsibilities:
1. Documentation Search
   - Search across supported products: 'kubernetes', 'istio', 'argo cd', 'argo rollouts', 'helm', 'prometheus'
   - Provide accurate, relevant information
   - Include documentation links

2. Information Analysis
   - Understand context of queries
   - Cross-reference multiple sources
   - Provide comprehensive answers

Guidelines:
- Always verify product context before searching
- Provide direct documentation links
- Explain complex concepts clearly
- Cross-reference related documentation

Process for Each Query:
1. Identify target product
2. Search relevant documentation
3. Analyze and synthesize information
4. Provide clear, documented response
"""

# Create agents with improved configurations
k8s_agent = AssistantAgent(
    "k8s_agent",
    description="Kubernetes specialist for cluster operations and resource management",
    tools=k8s_tools,
    model_client=model_client,
    system_message=K8S_SYSTEM_MESSAGE,
)

argo_agent = AssistantAgent(
    "argo_agent",
    description="Argo Rollouts specialist for progressive delivery management.",
    tools=argo_setup_tools,
    model_client=model_client,
    system_message=ARGO_SYSTEM_MESSAGE,
)

docs_agent = AssistantAgent(
    "cncf_docs_agent",
    description="CNCF documentation specialist for accurate information retrieval",
    tools=[QueryTool(config=docs_config)],
    model_client=model_client,
    system_message=DOCS_SYSTEM_MESSAGE,
)

team = SelectorGroupChat(
    [planning_agent, argo_agent, k8s_agent, docs_agent],
    model_client=model_client,
    termination_condition=TextMentionTermination("TERMINATE") | MaxMessageTermination(max_messages=25),
    allow_repeated_speaker=True,
)


# Example usage
async def run_team_task(task: str):
    """Run a task through the multi-agent team with proper error handling. If you don't have any explicit tasks left to complete, return TERMINATE."""
    try:
        return await Console(team.run_stream(task=task))
    except Exception as e:
        logging.error(f"Error executing task: {e}")
        return None


# Usage example:
# task = "Help setup the Argo Rollouts controller in my cluster?"
task = "Help me use Kubernetes Gateway APIs with argo rollouts in my cluster? What am I missing?"
await run_team_task(task)

---------- user ----------
Help me use Kubernetes Gateway APIs with argo rollouts in my cluster? What am I missing?
---------- PlanningAgent ----------
To help you use Kubernetes Gateway APIs with Argo Rollouts in your cluster, we need to ensure that the Argo Rollouts controller is installed and properly configured in your Kubernetes cluster. Then, we need to make sure that your cluster has the necessary components of the Kubernetes Gateway APIs available. Here are the subtasks involved:

1. argo_agent: Validate the installation of the Argo Rollouts controller in the Kubernetes cluster.
2. argo_agent: Check the configuration of the Argo Rollouts to ensure it is set up for your cluster environment properly.
3. k8s_agent: Verify if the necessary CRDs (Custom Resource Definitions) and resources for Kubernetes Gateway APIs are present in the cluster.

After these checks, we can summarize the findings and advise on any further steps or configurations needed.
---------- argo_agent ----------

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Help me use Kubernetes Gateway APIs with argo rollouts in my cluster? What am I missing?', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=218, completion_tokens=163), content='To help you use Kubernetes Gateway APIs with Argo Rollouts in your cluster, we need to ensure that the Argo Rollouts controller is installed and properly configured in your Kubernetes cluster. Then, we need to make sure that your cluster has the necessary components of the Kubernetes Gateway APIs available. Here are the subtasks involved:\n\n1. argo_agent: Validate the installation of the Argo Rollouts controller in the Kubernetes cluster.\n2. argo_agent: Check the configuration of the Argo Rollouts to ensure it is set up for your cluster environment properly.\n3. k8s_agent: Verify if the necessary CRDs (Custom Resource Definitions) and resources for Kubernetes Gateway APIs are present in 

In [ ]:
print(team.dump_component().model_dump_json(indent=2))